<a href="https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Datascraping con Twitter"


Uno de los superpoderes de la AI/ML es que nos permite entender el mundo desde la perspectiva de los datos.
Twitter es una plataforma que proporciona datos extensos en lenguaje natural, en base a los cuales se puede explorar mucho.

In [ ]:
# importing libraries
import pandas as pd
import tweepy
import time
import warnings
#warnings.filterwarnings('ignore')

import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

Para acceder a la API de Twitter, necesitaremos 4 elementos de la página de la aplicación de Twitter. Estas claves se encuentran en la configuración de la aplicación de Twitter en la pestaña Claves y tokens de acceso:

- consumer key
- consumer seceret key
- access token key
- access token secret key

[Detailed info for getting keys](https://www.slickremix.com/docs/how-to-get-api-keys-and-tokens-for-twitter/)

In [ ]:
consumer_key="PshLLH4ssju6vpZOxn318nAto"
consumer_secret="heVLch5djnTURRFpkKyZMkNs3zDMTbEkrbsvBbBm6chLQDuCz3"
access_token="6235072-p0KRSsEZuLLjNTHl4JcA7GIZ9qoW3X9hYUSsMxbFh5" #
access_token_secret="dq1vBftPcpU0iGOMQRZZxgGam2WfjU9syLdvmmPhmi9ZP" # 

In [ ]:
#@title Enter Twitter Credentials
TWITTER_KEY = 'PshLLH4ssju6vpZOxn318nAto' #@param {type:"string"}
TWITTER_SECRET_KEY = 'heVLch5djnTURRFpkKyZMkNs3zDMTbEkrbsvBbBm6chLQDuCz3' #@param {type:"string"}

In [ ]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [ ]:
#@title Twitter Search API Inputs
#@markdown ### Enter Search Query:
searchQuery = 'Qatar' #@param {type:"string"}
#@markdown ### Enter Max Tweets To Scrape:
#@markdown #### The Twitter API Rate Limit (currently) is 45,000 tweets every 15 minutes.
maxTweets = 200 #@param {type:"slider", min:0, max:45000, step:100}
Filter_Retweets = True #@param {type:"boolean"}

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

clear_output()
print("Downloaded {0} tweets".format(tweetCount))

Downloaded 200 tweets


In [ ]:
#Accessing twitter API
auth=tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token=(access_token, access_token_secret)
api= tweepy.API(auth, wait_on_rate_limit= True)

In [ ]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df.to_csv('tweets.csv')
tweet_df.head()

Un problema importante al que nos enfrentamos es que cuando convertimos una lista en un DataSet, si el texto del tweet tiene> 50 caracteres, se trunca. Para resolver este problema, debemos establecer el ancho de la columna de visualización para el dataset de datos de pandas en -1.

In [ ]:

pd.set_option('display.max_colwidth', -1)

## Obtener tweets de un usuario especifico



In [ ]:
#collapse_show
# Doing a function implementation of the tweets
# In this case if the tweet is retweeted we wont get full text
def get_tweets(username, count):
    try:
        #creating query methods using parameters
        tweets= tweepy.Cursor(api.user_timeline,id= username, tweet_mode="extended").items(count)
        tweet_list= [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
        #creating dataframe from tweets list
        tweets_df=pd.DataFrame(tweet_list, columns=["Date","Tweet_id","Tweet_Text"])
    
    except BaseException as e:
        print('failed on_status', str(e))
        time.sleep(3)
        
    return tweets_df


In [ ]:
df=get_tweets("USFQBSpregrado",10)

failed on_status Twitter error response: status code = 404


UnboundLocalError: ignored

## Obtener tweets de una palabra

In [ ]:
# Getting text from a  search string
text_query="#USFQ -filter:retweets"
count=10
try:
    #creation of query method using parameters
    tweets= tweepy.Cursor(api.search, q=text_query, tweet_mode="extended", lang='es').items(count)

    #getting the information from twitter object
    tweet_list= [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
    #creating a data frame from the list
    tweet_df_from_query= pd.DataFrame(tweet_list, columns=['Date','tweet_id', 'tweet_text'])
        
except BaseException as e:
    print('failed_on_status', str(e))
    time.sleep(3)

In [ ]:
tweet_df_from_query

,Date,tweet_id,tweet_text
0,2021-11-28 18:54:24,1465031337077874693,"𝐅𝐥𝐲𝐰𝐡𝐞𝐞𝐥 𝐩𝐚𝐫𝐚 𝐚𝐭𝐫𝐚𝐞𝐫 𝐲 𝐟𝐢𝐝𝐞𝐥𝐢𝐳𝐚𝐫 𝐜𝐥𝐢𝐞𝐧𝐭𝐞𝐬\n\n🗓 𝗟𝘂𝗻𝗲𝘀, 𝟮𝟵 𝗡𝗢𝗩 𝟮𝟬𝟮𝟭\n🕐 𝟭𝟯𝗵𝟬𝟬\n\n𝗜𝗻𝘃𝗶𝘁𝗮𝗱𝗮:\n👩🏻‍🏫 Paola Valencia, profesora del Business School #USFQ.\n\nTransmisión ➡️ Facebook Live de @ExponentialSet.\n\n¡Te esperamos! https://t.co/6ocPy5cL5c"
1,2021-11-26 23:12:11,1464371433463812099,"🐉 En este artículo, Martín González habla sobre el evento realizado por el club de marketing de la USFQ sobre el Neuromarketing. @MarketeersUSFQ #usfq \n\n🐉 Para más información accede al artículo completo en el link de la biografía. https://t.co/v0fY3sjIyc"
2,2021-11-26 21:50:57,1464350993538240514,Quito ha experimentado muchos temblores en su historia. Visita la muestra “A que no vuelva el temblor a sorprendernos furioso” en el Museo del Carmen Alto y conoce cómo han influido en la cultura religiosa de la ciudad.#tomorrowscities #USFQ #DRR @USFQ_Ecuador @MuseoCarmenAlto https://t.co/UrQpywWvLi
3,2021-11-26 16:22:40,1464268377854287872,"🐉 En Aula Magna nos preocupamos por tu bienestar, por eso hoy te traemos 8 consejos para el regreso a clases presenciales. #usfq #aulamagna #educación https://t.co/cIdUGffE4q"
4,2021-11-26 04:15:55,1464085482631876608,"Noche de premios, velada especial! \n\n#gratitud #graciasadios #graciasgraciasgracias #premioscocoa2021 #usfq #Quito en Ecuador https://t.co/ru4JpScyBJ"
5,2021-11-25 22:38:03,1464000457328795652,"🐉 Para este segmento de @usfqalumni entrevistamos a Alba María Cabezas, estudiante graduada en la USFQ, sobre su trayectoria profesional y uno de sus proyectos más importantes ‘Alana food and Play’.\n#orgullousfqalumni #usfq https://t.co/mwUKy5CJIt"
6,2021-11-25 02:46:54,1463700693492117512,"🌎 ¿Emergencia climática, un asunto de derechos humanos? \n\nEl derecho a un medio ambiente seguro, limpio, saludable y sostenible también es reconocido como un derecho humano y se debe hacer lo necesario para preservarlo. #usfq #medioambiente #derechoshumanos https://t.co/c9MUmlgVBk"
7,2021-11-25 01:29:07,1463681120235659267,El Centro de Conservación ex-situ 🐸🐸 más fenomenal de América del Sur! @arcasapos. Visita de la Maestría en Ecología #USFQ 🐉🇪🇨. https://t.co/Nn12DzVDhe https://t.co/SBlJqazdaU
8,2021-11-24 01:03:36,1463312309292572674,Caso 4: Investigación de estudiantes sobre el comportamiento del segmento de microcrédito y las alternativas de crédito en el Ecuador. \n\n#OikoDataClub #EconomíaIndustrial #Microcrédito #SistemaFinanciero #EconomíaIndustrial #Ecuador #Oikonomics #IIE #Econpuce #USFQ #UDLA #UCE. https://t.co/itPaZw66yK
9,2021-11-24 00:55:00,1463310144985571335,"@USFQ_Alumni @USFQ_Ecuador @MarketeersUSFQ @ClubFNZUSFQ @BHSUSFQ #USFQ Career Talk\n🗓️ 25 NOV 2021\n🕗 13h00\n🗣️ María Emilia Rivadeneira, Asuntos Corporativos y Sostenibilidad en @LATAMAirlines\n\nRegistro: https://t.co/2dlCK8GdiW\n\nCareer Talks: profesionales brindan consejos y comparten experiencias con nuestros estudiantes 🐲. https://t.co/jzqb6twajK"


-----

## Extracting trends of a perticular region

In [ ]:
trend=pd.DataFrame(api.trends_available())
str(trend['country'])
trend['country']="Ecuador"
print(trend)

            name                          placeType  ...     woeid  countryCode
0    Worldwide    {'code': 19, 'name': 'Supername'}  ...  1         None       
1    Winnipeg     {'code': 7, 'name': 'Town'}        ...  2972      CA         
2    Ottawa       {'code': 7, 'name': 'Town'}        ...  3369      CA         
3    Quebec       {'code': 7, 'name': 'Town'}        ...  3444      CA         
4    Montreal     {'code': 7, 'name': 'Town'}        ...  3534      CA         
..        ...                             ...        ...   ...      ..         
462  Vietnam      {'code': 12, 'name': 'Country'}    ...  23424984  VN         
463  Petaling     {'code': 7, 'name': 'Town'}        ...  56013632  MY         
464  Hulu Langat  {'code': 7, 'name': 'Town'}        ...  56013645  MY         
465  Ahsa         {'code': 9, 'name': 'Unknown'}     ...  56120136  SA         
466  Okayama      {'code': 7, 'name': 'Town'}        ...  90036018  JP         

[467 rows x 7 columns]


In [ ]:
[i['name'] for i in api.trends_place(id=23424848)[0]['trends']]

TweepError: ignored

----
